In [34]:
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import scipy
from matplotlib.colors import ListedColormap
from functools import partial
from sklearn.model_selection import KFold
import numpy as np

In [35]:
df = pd.read_csv('wine.data', header=None, delimiter=',', dtype=float)
df = df.drop([2,3,4,5,6,8,9,10,11,12,13], axis=1)
df = df[df[0] != 3.0]
df[0] = df[0].replace({1.0: 1, 2.0: -1})
df[0] = df[0].astype(float)
X = df.iloc[:, 1:3].values  # Selects columns 2 and 3
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X) # THIS REALLY HELPED
Y = df.iloc[:, 0].values    # Selects column 1
Y = Y.reshape(-1, 1).astype(float)  
X_and_Y = np.hstack((X, Y))     # Stack them together for shuffling.
np.random.seed(1)               # Set the random seed.
np.random.shuffle(X_and_Y)      # Shuffle the data points in X_and_Y array

print(X.shape)
print(Y.shape)
print(X_and_Y[0])

(130, 2)
(130, 1)
[-0.81784764 -0.61230964 -1.        ]


In [36]:
def vis(X, Y, knn_classifier=None):
    # Visualize k-NN.
    if knn_classifier is not None:
        # Calculate min, max and create grids.
        h = .02
        x0_min, x0_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
        x1_min, x1_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
        x0s, x1s = np.meshgrid(np.arange(x0_min, x0_max, h),
                               np.arange(x1_min, x1_max, h))
        xs = np.stack([x0s, x1s], axis=-1).reshape(-1, 2)

        # Predict class using kNN classifier and data.
        ys_pred = np.array([knn_classifier(x) for x in xs])
        ys_pred = ys_pred.reshape(x0s.shape)

        # Put the result into a color plot.
        # Color map: #00AAFF - blue, #FFAAAA - red, #AAFFAA - green

        cmap_light = ListedColormap(['#00AAFF', '#FFAAAA'])
        plt.pcolormesh(x0s, x1s, ys_pred, cmap=cmap_light, alpha=0.3)

    indices_neg1 = (Y == -1).nonzero()[0]
    indices_pos1 = (Y == 1).nonzero()[0]
    plt.scatter(X[:,0][indices_neg1], X[:,1][indices_neg1],
                c='blue', label='class -1', alpha=0.3)
    plt.scatter(X[:,0][indices_pos1], X[:,1][indices_pos1],
                c='red', label='class +1', alpha=0.3)
    plt.legend()
    plt.xlabel('$x_0$')
    plt.ylabel('$x_1$')

    plt.show()

In [37]:
# Euclidean distance.
def calc_distance(x1, x2):
    # x1, x2 are two NumPy vectors
    # Return the Euclidean distance between x1 and x2. It should be a scalar.
    dist = np.sqrt(np.sum((x1-x2)**2))
    return dist

# k nearest neighbor predictor.
def f_knn(x, X_train, Y_train, k):
    # Create the list of (distance, label) pairs.
    dist_label_pairs = []
    for xi, yi in zip(X_train, Y_train):
        # Calculate the distance.
        dist = calc_distance(xi, x)
        # Add a (distance, label) pair to the list.
        dist_label_pairs.append((dist, yi))
    # Sort the pairs by distance (ascending).
    sorted_dist_label_pairs = sorted(dist_label_pairs, key=lambda x:x[0])
    # Obtain the first k pairs (corresponding to k smallest distances).
    k_dist_label_pairs = sorted_dist_label_pairs[:k]
    # Extract the labels of the k pairs.
    k_labels = [label for _, label in k_dist_label_pairs]
    # Count the number of +1 predictions and -1 predictions.
    pos1_in_k_labels = 0
    neg1_in_k_labels = 0
    for label in k_labels:
        if label == +1:
            pos1_in_k_labels += 1
        elif label == -1:
            neg1_in_k_labels += 1
    # Make the prediction based on counts.
    if pos1_in_k_labels > neg1_in_k_labels:
        y_pred = +1
    else:
        y_pred = -1

    return y_pred

# Judge function: 1(a != b). It supports scalar, vector and matrix.
def judge(a, b):
    return np.array(a != b).astype(np.float32)

# Calculate error given feature vectors X and labels Y.
def calc_error(X, Y, knn_classifier):
    e = 0
    n = len(X)
    for (xi, yi) in zip(X, Y):
        e += judge(yi, knn_classifier(xi))
    e = 1.0 * e / n
    return e

# KNN
# 20 training

In [45]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.2 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(26, 2)
(26,)
(13, 2)
(13,)
(91, 2)
(91,)


In [53]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.0

k=2
Validation error: 0.0

k=3
Validation error: 0.07692307692307693

k=4
Validation error: 0.07692307692307693

k=5
Validation error: 0.07692307692307693

k=6
Validation error: 0.07692307692307693

k=7
Validation error: 0.07692307692307693

k=8
Validation error: 0.0

Best k=1
Train error:0.0
Test error: 0.08791208791208792


# 50 training

In [54]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.5 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(65, 2)
(65,)
(13, 2)
(13,)
(52, 2)
(52,)


In [55]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.0

k=2
Validation error: 0.0

k=3
Validation error: 0.0

k=4
Validation error: 0.0

k=5
Validation error: 0.0

k=6
Validation error: 0.0

k=7
Validation error: 0.0

k=8
Validation error: 0.0

Best k=1
Train error:0.0
Test error: 0.09615384615384616


# 80 training

In [56]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.8 * len(X_shuffled))  # 20% for training
val_split = train_split + int(0.1 * len(X_shuffled))  # Additional 20% for validation

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_val = X_shuffled[train_split:val_split]
Y_val = Y_shuffled[train_split:val_split]
X_test = X_shuffled[val_split:]
Y_test = Y_shuffled[val_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)


(104, 2)
(104,)
(13, 2)
(13,)
(13, 2)
(13,)


In [57]:
opt_val_error = 1.0
opt_k = None

# Try different k(s).
for k in [1,2,3,4,5,6,7,8]:
    # Visualize
    #   1. Validation set (as points).
    #   2. Decision boundary from training set (as background).
    print("k={}".format(k))
    # Create a k-NN classifier with training set.
    knn_classifier = partial(f_knn, X_train=X_train, Y_train=Y_train, k=k)
    # Visualization.
    #vis(X_train, Y_train, knn_classifier)
    # Calculate validation error.
    val_error = calc_error(X_val, Y_val, knn_classifier)
    print("Validation error: {}\n".format(val_error))
    if val_error < opt_val_error:
        opt_val_error = val_error
        opt_k = k
        opt_knn_classifier = knn_classifier
print("Best k={}".format(opt_k))
train_error = calc_error(X_train,Y_train,opt_knn_classifier)
print("Train error:{}".format(train_error))
test_error = calc_error(X_test, Y_test, opt_knn_classifier)
#vis(X_test, Y_test, knn_classifier)
print("Test error: {}".format(test_error))

k=1
Validation error: 0.07692307692307693

k=2
Validation error: 0.15384615384615385

k=3
Validation error: 0.0

k=4
Validation error: 0.0

k=5
Validation error: 0.0

k=6
Validation error: 0.0

k=7
Validation error: 0.0

k=8
Validation error: 0.0

Best k=3
Train error:0.038461538461538464
Test error: 0.23076923076923078
